In [1]:
import nltk 
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from gensim.models import FastText
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_vectors
import gensim
import math
import random
import numpy as np
from unidecode import unidecode

In [2]:
questions_train = []
tables_train = []
actual_col_train = []
label_cols_train = []
label_rows_train = []
# Read JSON data from file line by line
with open('data/A2_train.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Parse JSON data from each line
        parsed_data = json.loads(line)
        questions_train.append(parsed_data['question'])
        tables_train.append(parsed_data['table'])
        label_cols_train.append(parsed_data['label_col'][0])
        actual_col_train.append(list(parsed_data['table']['cols']))
        label_rows_train.append(list(parsed_data['label_row']))

# questions_train = clean_questions(questions_train)

print('Number of questions:', len(questions_train))
print('Number of tables:', len(tables_train))
print('Number of label columns:', len(label_cols_train))
print('Number of actual columns:', len(actual_col_train))
print('Number of label rows:', len(label_rows_train))

questions_test = []
tables_test = []
actual_col_test = []
label_cols_test = []
label_rows_test = []
qid_test = []
# Read JSON data from file line by line
with open('data/A2_val.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Parse JSON data from each line
        parsed_data = json.loads(line)
        questions_test.append(parsed_data['question'])
        tables_test.append(parsed_data['table'])
        label_cols_test.append(parsed_data['label_col'][0])
        actual_col_test.append(list(parsed_data['table']['cols']))
        qid_test.append(parsed_data['qid'])
        label_rows_test.append(list(parsed_data['label_row']))

# questions_test = clean_questions(questions_test)

print('Number of questions:', len(questions_test))
print('Number of tables:', len(tables_test))
print('Number of label columns:', len(label_cols_test))
print('Number of actual columns:', len(actual_col_test))
print('Number of qids is ', len(qid_test))
print('Number of label rows:', len(label_rows_test))


Number of questions: 25000
Number of tables: 25000
Number of label columns: 25000
Number of actual columns: 25000
Number of label rows: 25000
Number of questions: 5000
Number of tables: 5000
Number of label columns: 5000
Number of actual columns: 5000
Number of qids is  5000
Number of label rows: 5000


In [20]:
predictions_train = []
for i in range(len(questions_train)):
    question = unidecode(questions_train[i]).lower()
    table = tables_train[i]
    actual_col = actual_col_train[i]
    label_col = label_cols_train[i][0]
    label_row = label_rows_train[i]
    predicted_row = []
    score_of_row = []
    for j in range(len(table['rows'])):
        row = table['rows'][j]
        score = 0.0
        for k in range(len(row)):
            cell = unidecode(row[k]).lower()
            that_col = unidecode(actual_col[k]).lower()
            if cell in question:
                if that_col in question:
                    distance = abs(question.index(cell) - question.index(that_col))
                    score += (43.0)*(len(cell)*len(cell))/(distance+1)
                else:
                    score += len(cell)*len(cell)
        score_of_row.append(score)
    
    max_score = max(score_of_row)
    for i in range(len(score_of_row)):
        if score_of_row[i] == max_score:
            predicted_row.append(i)
    predictions_train.append(predicted_row)

print('Number of predictions:', len(predictions_train))

Number of predictions: 25000


In [21]:
correct = 0
for i in range(len(predictions_train)):
    if(predictions_train[i] == label_rows_train[i]):
        correct += 1
    # else:
        # print(questions_train[i])
        # print(tables_train[i])
accuracy = correct/len(predictions_train)
print('Accuracy on train data is:', accuracy)

Accuracy on train data is: 0.88752


In [76]:
predictions_test = []
for i in range(len(questions_test)):
    question = questions_test[i].lower()
    table = tables_test[i]
    actual_col = actual_col_test[i]
    label_col = label_cols_test[i][0]
    label_row = label_rows_test[i]
    predicted_row = []
    score_of_row = []
    for j in range(len(table['rows'])):
        row = table['rows'][j]
        score = 0.0
        for k in range(len(row)):
            cell = row[k].lower()
            that_col = actual_col[k].lower()
            if cell in question:
                if that_col in question:
                    distance = abs(question.index(cell) - question.index(that_col))
                    score += (50.0)*(len(cell)*len(cell))/(distance+1)
                else:
                    score += len(cell)*len(cell)
        score_of_row.append(score)
    
    max_score = max(score_of_row)
    for i in range(len(score_of_row)):
        if score_of_row[i] == max_score:
            predicted_row.append(i)
    predictions_test.append(predicted_row)

print('Number of predictions:', len(predictions_test))

        

Number of predictions: 5000


In [77]:
correct = 0
for i in range(len(predictions_test)):
    if(predictions_test[i] == label_rows_test[i]):
        correct += 1
accuracy = correct/len(predictions_test)
print('Accuracy on test data is:', accuracy)

Accuracy on test data is: 0.886
